<a href="https://colab.research.google.com/drive/1Yn6YzRW-S8mxB8773lfcUzbd6cmudQRy?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Python GPU Programming: Estimating Pi with Monte Carlo Simulation


In [ ]:
! pip install pyopencl mako -q

In [20]:
# NumPy Pi Estimation with Monte Carlo Simulation
import numpy as np
import time

t0 = time.time()

n_runs = 10**8

# Simulate Random Coordinates in Unit Square:
ran = np.random.uniform(low=-1, high=1, size=(2, n_runs))

# Identify Random Coordinates that fall within Unit Circle and count them
result = ran[0]**2 + ran[1]**2 <= 1
n_in_circle = np.sum(result)

# Estimate Pi
print("Pi Estimate: ", 4 * n_in_circle / n_runs)

print("Time Elapsed: ", time.time() - t0)

Pi Estimate:  3.14179664
Time Elapsed:  3.100929021835327


## Several GPU Approaches

### Create Context and Command Queue

In [4]:
import pyopencl as cl
import pyopencl.clrandom as clrand
import pyopencl.array as cl_array
from pyopencl.elementwise import ElementwiseKernel
from pyopencl.reduction import ReductionKernel
import time
import numpy as np
 
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

### Map on GPU, Reduce on CPU

In [35]:
t0 = time.time()
n_runs = 10**8

x_dev = clrand.rand(queue, (n_runs), np.float32, a=-1, b=1)
y_dev = clrand.rand(queue, (n_runs), np.float32, a=-1, b=1)

map_result = (y_dev ** 2 + x_dev ** 2) <= 1
n_total = map_result.get()
n_in = np.sum(n_total)

print(4 * n_in / n_runs)
print("Time Elapsed (Map on GPU): ", time.time() - t0)

3.14161396
Time Elapsed (Map on GPU):  0.23005914688110352



### Combined Map/Reduce on GPU

In [37]:
t1 = time.time()
n_runs = 10**8
x_dev = clrand.rand(queue, (n_runs), np.float32, a=-1, b=1)
y_dev = clrand.rand(queue, (n_runs), np.float32, a=-1, b=1)

rknl = ReductionKernel(ctx, np.float32,
        arguments="float *x, float *y",
        map_expr="(x[i]*x[i] + y[i]*y[i]) <= 1 ? 1 : 0",
        reduce_expr="a+b",
        neutral="0",
        )

n_in = rknl(x_dev, y_dev).get()
print(4 * n_in / n_runs)
 
print("Time Elapsed (Map + Reduce on GPU): ", time.time() - t1)

3.1415648
Time Elapsed (Map + Reduce on GPU):  0.018733501434326172


## Bonus: Abstracting Away Explicit Memory Operations with `numba`

In addition to compiling code for CPUs, `numba` also gives us the ability to compile (rudimentary) code for GPUs via similar decorators. Here, we don't need to explicitly deal with transferring data to and from the GPU if we don't want to (although, we will not see the same speedups that PyOpenCL/PyCUDA will give us). Under the hood, `numba` uses CUDA, but we do not need to explicitly write code using CUDA syntax.

In [13]:
from numba import vectorize, cuda
from numba.core.errors import NumbaPerformanceWarning
import warnings
warnings.simplefilter('ignore', category=NumbaPerformanceWarning)

# `numba` has GPU RNG, but only compare map/reduce ops here
# see here for example of GPU-side RNG: https://numba.readthedocs.io/en/stable/cuda/random.html
n_runs = 10**8
ran = np.random.uniform(low=-1, high =1, size=(2, n_runs)).astype(np.float32)
x, y = ran[0], ran[1]

@vectorize(['i4(f4, f4)'], target='cuda')
def in_circle(x, y):
  '''
  Vectorized function takes in x, y coordinates (float32, float32) within an
  array and returns a boolean indication of whether these values are (1) or
  are not (0) in the unit circle (int32). All computation is done on the GPU.
  '''
  in_circle = x**2 + y**2 <= 1
  return in_circle

@cuda.reduce
def gpu_sum(a, b):
  '''
  Sums values in an array together on the GPU.
  '''
  return a + b

First, we might try to only perform the mapping operation on the GPU (passing the x and y arrays over to the GPU, computing whether the random coordinates are in the unit circle or not, and then sending values back to the CPU to sum and estimate pi).

Note that this is in the same order of magnitude as the GPU Map/CPU Reduce solution above with PyOpenCL (although PyOpenCL/PyCUDA can be a bit faster -- note the times above).

In [18]:
%%timeit
result = in_circle(x, y)
4 * np.sum(result) / n_runs

397 ms ± 99.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Can also perform the summation on the GPU...

In [17]:
%%timeit
# Perform both in_circle check and sum ops on GPU
result = in_circle(x, y)
4 * gpu_sum(result) / n_runs

298 ms ± 16.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


And store intermediate results on the GPU (similar to the `reduce` kernel defined in PyOpenCL above, but adding in some extra complexity):

In [16]:
%%timeit
# Slight improvement by keeping intermediate result on GPU, but not to the
# degree of PyOpenCL's fast map + reduce. Would need to define custom kernel
# to match PyOpenCL/PyCUDA's speed with `numba`
# (outside the scope of the course: https://numba.pydata.org/numba-doc/latest/cuda/kernels.html)
in_circle_dev = cuda.device_array(shape=(n_runs,),
                                  dtype=np.float32)
in_circle(x, y, out=in_circle_dev)
4 * gpu_sum(in_circle_dev) / n_runs

211 ms ± 23.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
